In [ ]:
# load data and processing


In [ ]:
import numpy as np
import scipy as sp
from scipy.linalg import norm
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy.matlib
import pandas as pd

from allensdk.api.queries.ontologies_api import OntologiesApi
from allensdk.core.mouse_connectivity_cache import MouseConnectivityCache
from mcmodels.core import VoxelModelCache

import pickle

import networkx as nx
import plotly.offline as py
import plotly.graph_objects as go
import re

import sklearn as sk
# import plotly.express as px

In [ ]:
import plotly.express as px
mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['ps.fonttype'] = 42
plt.rcParams['svg.fonttype'] = 'none'

In [ ]:
# load helper functions
from helper import load_hierarchy, plot_connectivity, generate_connectivity, generate_random_connectivity
from helper import load_celldensity, load_interneurondensity, interneuronDensityProcessing
import modelHelper

In [ ]:
filepath = './'  # change path accordingly
libfilepath = './lib/'  # change path accordingly

# load the harris hierarchy N =37; 
area_list_hier = ['ACAd', 'ACAv', 'AId', 'AIp', 'AIv', 'AUDd', 'AUDp', 'AUDpo', 'FRP', 'ILA', 'MOp', 'MOs', 'ORBl',
         'ORBm', 'ORBvl', 'PL', 'RSPagl', 'RSPd', 'RSPv', 'SSp-bfd', 'SSp-ll', 'SSp-m', 'SSp-n',
         'SSp-tr', 'SSp-ul', 'SSs', 'TEa', 'VISa', 'VISal', 'VISam', 'VISl', 'VISli', 'VISp',
         'VISpl', 'VISpm', 'VISpor', 'VISrl']  # list of areas in the current model based on published paper.
        # SSp-un and VISC are removed. 

hierfilename = 'mouse_harrishier_extracted_CC_published.csv'

[harrisHierarchy, harrisHierarchy_df] = load_hierarchy(libfilepath, hierfilename, area_list_hier)
harrisHierarchy_df.to_csv('results/dfHarrisHier.csv')

In [ ]:
harrisHierarchy_df

In [ ]:
area_list = ['ACAd', 'ACAv', 'AId', 'AIp', 'AIv', 'AUDd', 'AUDp', 'AUDpo', 'AUDv', 'ECT', 'FRP', 'GU', 'ILA', 'MOp', 'MOs', 'ORBl',
         'ORBm', 'ORBvl', 'PERI', 'PL', 'RSPagl', 'RSPd', 'RSPv', 'SSp-bfd', 'SSp-ll', 'SSp-m', 'SSp-n',
         'SSp-tr', 'SSp-ul', 'SSp-un', 'SSs', 'TEa','VISC', 'VISa', 'VISal', 'VISam', 'VISl', 'VISli', 'VISp',
         'VISpl', 'VISpm', 'VISpor', 'VISrl']
n_areas = len(area_list)

In [ ]:
# define divisions from Harris et al.
div = {'somatomotor':['SSp-bfd','SSp-tr','SSp-ll','SSp-ul',
                     'SSp-un','SSp-n','SSp-m','SSs','MOp','MOs'],
      'visual':['VISal','VISl','VISp','VISpl',
                'VISli','VISpor','VISrl'],
      'medial':['VISa','VISam','VISpm','RSPagl',
               'RSPd','RSPv'],
      'prefrontal':['FRP','ACAd','ACAv','PL',
                   'ILA','ORBl','ORBm','ORBvl'],
      'lateral':['AId','AIv','AIp','VISC',
                 'GU','TEa','PERI','ECT'],
      'auditory':['AUDd','AUDp','AUDpo','AUDv']}

div_color_list = np.array([[255, 140, 0],
                           [169, 164, 217],
                           [22, 37, 128],
                           [220,20,60],
                           [218,165,32],
                           [91,  0,  105]])/255
# div_color_list = {
#     'somatomotor':[255, 140, 0],
#       'visual':[169, 164, 217],
#       'medial':[22, 37, 128],
#       'prefrontal':[220,20,60],
#       'lateral':[218,165,32],
#       'auditory':[91,  0,  105]}

div_name_list = ['somatomotor',
                 'visual',
                 'medial',
                 'prefrontal',
                 'lateral',
                 'auditory']

In [ ]:
def plotHiernotebook(hierarchy_df, dotSize, yaxis, figureSize, fontSize, saveFig, fileName):
        # seaborn.set()
    dfRaw = hierarchy_df.sort_values(hierarchy_df.columns[0])

    fig = plt.figure(figsize = figureSize, dpi=300, facecolor='w', edgecolor='k')
    plt.rcParams.update({'font.size': fontSize})
    
    xx = np.arange(len(hierarchy_df))
    plt.plot(xx, dfRaw.values, '.-', markersize=dotSize, color='k')
    plt.xticks(xx, dfRaw.index, rotation = 90, fontsize=fontSize*0.6)
    plt.ylabel(yaxis) #r'$e_i/e_1$'
    # regPV = sk.linear_model.LinearRegression()
    # xx2 = xx[np.newaxis].T
    # regPV.fit(xx2, dfRaw.values)
    # print(regPV.coef_, regPV.intercept_)
    # plt.plot(xx, xx * regPV.coef_[0][0] + regPV.intercept_[0], alpha = 0.5)
    ax = plt.gca()
#     [t.set_color('red') for t in ax.xaxis.get_ticklabels()]
    for t in ax.xaxis.get_ticklabels():
        label = t._text
        for i in range(6):
            if label in div[div_name_list[i]]:
                t.set_color(div_color_list[i])
    # plt.grid(alpha = 0.5)

    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    if saveFig:
        plt.savefig('figure/' + fileName, dpi=300, bbox_inches='tight',transparent=True)

In [ ]:
plotHiernotebook(harrisHierarchy_df, 8, r'Harris Hierarchy', (5,3), 12, True, 'loadDataProcessing/harrisHierarchyPlotnotebook.svg')

In [ ]:
# load connectivity matrix
mcc = MouseConnectivityCache(manifest_file='connectivity/mouse_connectivity_manifest.json')
# The manifest file is a simple JSON file that keeps track of all of
# the data that has already been downloaded onto the hard drives.
# If you supply a relative path, it is assumed to be relative to your
# current working directory.

# grab the StructureTree instance
structure_tree = mcc.get_structure_tree()

oapi = OntologiesApi()

# get the ids of all the structure sets in the tree
# structure_set_ids = structure_tree.get_structure_sets()

# download and cache the latest voxel model
# this method returns a tuple with object types:
# (VoxelConnectivityArray, Mask, Mask)
cache = VoxelModelCache(manifest_file='connectivity/voxel_model_manifest.json')

# extracting the cortical subnetwork. get the normalized connectvity density for ipsi side.
normalized_connection_density = cache.get_normalized_connection_density()
full_matrix = normalized_connection_density['ipsi']  # .loc[cortex_ids_int][cortex_ids_str]
mat = np.array(full_matrix)

# cortex
cortex_structures = structure_tree.get_structures_by_set_id([688152357]) # first get the full info of cortex area
# cortex_structures_refined = []
# for s in cortex_structures:
#     if s['id'] in full_matrix.index:
#         cortex_structures_refined.append(s)
cortex_ids_int = [s['id'] for s in cortex_structures]  # get the cortex area id
cortex_ids_str = [str(s['id']) for s in cortex_structures]  
cortex_acr = [s['acronym'] for s in cortex_structures]
cortex_names = [s['name'] for s in cortex_structures]

#thalamus
thal_structures = structure_tree.get_structures_by_set_id([688152363])
thal_structures_refined = [] # some of the thalmic structure not in the cconnectivity matirx
for s in thal_structures:
    if s['id'] in full_matrix.index:
        thal_structures_refined.append(s)

thal_ids_int = [s['id'] for s in thal_structures_refined] # store as a int
thal_ids_str = [str(s['id']) for s in thal_structures_refined] # store as a str
thal_acr = [s['acronym'] for s in thal_structures_refined]
thal_names = [s['name'] for s in thal_structures_refined]


cortical_area_number = np.size(cortex_acr)
print(cortical_area_number)

# W = mat[0:cortical_area_number, 0:cortical_area_number]  # extract the cortical areas

W_ = full_matrix.loc[cortex_ids_int][cortex_ids_str] # this gives the cortico cortical connection
W_cxth_ = full_matrix.loc[cortex_ids_int][thal_ids_str] # this gives cx to thal connecitions shape (43, 40)
W_thcx_ = full_matrix.loc[thal_ids_int][cortex_ids_str] # this gives thal to cx connecitions shape (40, 43)
W = np.array(W_).transpose() # ake a transpose of the W since what the paper used in the transponsed version.
W_cxth = np.array(W_cxth_).transpose()
W_thcx = np.array(W_thcx_).transpose()

In [ ]:
# not include RT
print(thal_acr[34])
thal_acr.remove('RT')
plt.imshow(W_cxth)
W_cxth = np.delete(W_cxth, 34, 0)
print(W_cxth.shape)
W_thcx = np.delete(W_thcx, 34, 1)
print(W_thcx.shape)

# save thalamic arealist
print(len(thal_acr))
np.savetxt("results/thal_list.csv", thal_acr, delimiter=",", fmt = '%s')


In [ ]:
# peek on the connectivity
plot_connectivity(W, cortex_acr,savefig=True)
plt.figure()
plt.imshow(W>1e-4)
W>0 / (len(W)**2 - len(W))

In [ ]:
    # areaidxlist = np.arange(len(cortex_acr))
    # # extract the connectivity for list of areas.
    # W_ = W[areaidxlist, :][:, areaidxlist]

In [ ]:
plt.imshow(W)


In [ ]:
# generate connectivity matrix 
# this function also sorted the cortical areas in W_cxth and W_thcx.
conn_cxcx,conn_cxth, conn_thcx, area_list_sort, thal_list_sort = generate_connectivity(
    W, cortex_acr, W_cxth, W_thcx, thal_acr ,imgout=False)

In [ ]:
print(area_list)

In [ ]:
# # quick check which thalamic area project to cortical area  # SSp-n 26  MOs 14 
# np.set_printoptions(precision=3, suppress=True)
# print(conn_thcx[14, :]*1e5) 

In [ ]:
# load the volume data and make processing.
dataFile = 'CCFv3SuppTable2.csv'
df = pd.read_csv(dataFile, header=1)
df.head()

In [ ]:
volumeList = []
for area in area_list:
    volume = float(df[df['abbreviation'] == area]['total_voxel_counts (10 um)'])
    volumeList.append(volume)
volume_df = pd.DataFrame([x for x in zip(area_list, volumeList)], columns=['area', 'volume'])
print(volume_df['volume'].max()/volume_df['volume'].min())
volume_df.head()



In [ ]:
fig = px.line(volume_df, 'area', 'volume')
fig.show()
fig.write_image("figure/loadDataProcessing/areaVolume.svg", width=1000, height=350, scale=1)

In [ ]:
# load layer 23 cell density
# rawl23density_df = pd.read_csv('results/l23cell_density_iso.csv')
# rawl23density_df.head()

In [ ]:
# # generate connectivity that is normalized by volume   and using layer 23 cell density
# areaV3toV2 = {'VISa':'PTLp', 'FRP':'PL', 'VISli':'VISl', 
#         'VISrl':'PTLp', 'VISpor':'VISpl', 'SSp-un': 'SSp-ul'}  # copied from helper.py 
# l23densityList = []
# for area in area_list: 
# #     if curRow.empty: 
#     if area in areaV3toV2: 
#         curRow = rawl23density_df.loc[rawl23density_df['Acronym'] == areaV3toV2[area]] 
#     else:
#         curRow = rawl23density_df.loc[rawl23density_df['Acronym'] == area] 
#     l23densityList.append(float(curRow['Neurons [mm-3]'])) 
# l23density_df = pd.DataFrame([x for x in zip(area_list, l23densityList)], 
#                           columns=['area', 'l23 neuron density'])

# l23density_df.head()

In [ ]:
# load cell density  paper: A cell atlas for the mouse brain.
rawdensityMarkram_df = pd.read_csv('results/cell_density_iso_Markram.csv')
rawdensityMarkram_df.head()

In [ ]:
#generate connectivity that is normalized by volume and norm by cell density
areaV3toV2 = {'VISa':'PTLp', 'FRP':'PL', 'VISli':'VISl', 
        'VISrl':'PTLp', 'VISpor':'VISpl', 'SSp-un': 'SSp-ul'}  # copied from helper.py 
densityMarkramList = []
exdensityMarkramList = []
inhdensityMarkramList = []
for area in area_list: 
#     if curRow.empty: 
    if area in areaV3toV2: 
        curRow = rawdensityMarkram_df.loc[rawdensityMarkram_df['Acronym'] == areaV3toV2[area]] 
    else:
        curRow = rawdensityMarkram_df.loc[rawdensityMarkram_df['Acronym'] == area] 
    densityMarkramList.append(float(curRow['Neurons'])) 
    exdensityMarkramList.append(float(curRow['Excitatory']))
    inhdensityMarkramList.append(float(curRow['Inhibitory']))
densityMarkram_df = pd.DataFrame([x for x in zip(area_list, densityMarkramList, exdensityMarkramList, inhdensityMarkramList)], 
                          columns=['area', 'neuron density', 'excitatory neuron density', 'inhibitory neuron density']) 

densityMarkram_df.head() 

In [ ]:
fig = px.line(densityMarkram_df, 'area', 'neuron density')
fig.show()
fig.write_image("figure/loadDataProcessing/neuronDensity.svg", width=1000, height=350, scale=1)

densityMarkram_df['ratio'] = densityMarkram_df['excitatory neuron density'] / densityMarkram_df['inhibitory neuron density']
fig = px.line(densityMarkram_df, 'area', ['ratio'])
fig.show()

In [ ]:
# seems not used. 
# load cell density
newrawdensity_df = pd.read_csv('results/newrawcell_density_iso.csv')
newrawdensity_df.head()

In [ ]:
# This is also from paper: A cell atlas for the mouse brain.   
rawdensity_df = pd.read_csv('results/cell_density_iso.csv')  # cell_density_iso.csv is same as cell_density_iso_Markram.csv
rawdensity_df.head()

areaV3toV2 = {'VISa':'PTLp', 'FRP':'PL', 'VISli':'VISl', 
        'VISrl':'PTLp', 'VISpor':'VISpl', 'SSp-un': 'SSp-ul'}  # copied from helper.py 
densityList = []

for area in area_list: 
#     if curRow.empty: 
    if area in areaV3toV2: 
#         print(rawdensity_df[rawdensity_df['Acronym'] == areaV3toV2[area]])
        curRow = rawdensity_df.loc[rawdensity_df['Acronym'] == areaV3toV2[area]]   
#         curRow = rawdensity_df.loc[]
    else:
        curRow = rawdensity_df.loc[rawdensity_df['Acronym'] == area] 
    densityList.append(float(curRow['Neurons'])) 
density_df = pd.DataFrame([x for x in zip(area_list, densityList)], 
                          columns=['area', 'neuron density']) 

density_df.head() 

In [ ]:
fig = px.line(density_df, 'area', 'neuron density')
fig.show()


In [ ]:
# # plot layer 23 density
# fig = px.line(l23density_df, 'area', 'l23 neuron density')
# fig.show()

In [ ]:
# modify the connectivity
# mm11
normDensity = densityList / np.mean(densityList)
densityMat = np.matlib.repmat(normDensity, len(normDensity), 1)

# normVolume = np.array(volume_df['volume'] / volume_df['volume'].mean())
normVolume = volumeList / np.mean(volumeList)
volumeMat = np.matlib.repmat(normVolume, len(normVolume), 1)

# conn_cxcx_densityNorm = conn_cxcx / (densityMat * densityMat.T)
# conn_cxcx_densityNorm = conn_cxcx / densityMat.T   # just for normalize based on target area density.
# conn_cxcx_densityNorm = conn_cxcx * volumeMat / densityMat.T   # normalize based on target area density and by volume
conn_cxcx_norm = conn_cxcx * volumeMat / densityMat.T # scaling based on volume

# conn_cxcx = conn_cxcx_norm

In [ ]:
# sum(conn_cxcx[0, :] * hierarchy.T.squeeze()) / sum(conn_cxcx[0, :]), hierarchy[1][0]
plt.imshow(conn_cxcx_norm)
plt.colorbar()

In [ ]:
# get symmetrical connectivity
sym_cxcx = 0.5 * (conn_cxcx + conn_cxcx.T)
anti_cxcx = 0.5 * (conn_cxcx - conn_cxcx.T)
sym = (norm(sym_cxcx) - norm(anti_cxcx)) / (norm(sym_cxcx) + norm(anti_cxcx))
print(sym) # if sym ==1 or -1, then conn_cxcx is symmetric. 
px.imshow(sym_cxcx, color_continuous_scale='viridis')
# conn_cxcx_norm
sym_cxcx_norm = 0.5 * (conn_cxcx_norm + conn_cxcx_norm.T)
px.imshow(sym_cxcx_norm, color_continuous_scale='viridis')

In [ ]:
noHierList = [8,9, 11,18,29,32]
print([area_list[i] for i in noHierList])
h = harrisHierarchy[:, 0]
print(h)
hierarchy = []
j = 0
for i in range(n_areas):
    if i in noHierList:
        hierarchy.append(0)
    else:
        hierarchy.append(h[j])
        j += 1
hierarchy = np.array(hierarchy)
print(hierarchy)

In [ ]:
sym_cxcx = 0.5 * (conn_cxcx + conn_cxcx.T)
anti_cxcx = 0.5 * (conn_cxcx - conn_cxcx.T)
sym = (norm(sym_cxcx) - norm(anti_cxcx)) / (norm(sym_cxcx) + norm(anti_cxcx))
print(sym) # if sym ==1 or -1, then conn_cxcx is symmetric. 
px.imshow(sym_cxcx, color_continuous_scale='viridis')
# conn_cxcx_norm
sym_cxcx_norm = 0.5 * (conn_cxcx_norm + conn_cxcx_norm.T)
px.imshow(sym_cxcx_norm, color_continuous_scale='viridis')

In [ ]:
conn_withHier = np.zeros((n_areas, n_areas))
conn_withHier_norm = np.zeros((n_areas, n_areas))
for i in range(n_areas):
    for j in range(n_areas):
        if j in noHierList:
            continue
        conn_withHier[i, j] = sym_cxcx[i, j]
#         conn_withHier[i, j] = conn_cxcx[j, i]
        conn_withHier_norm[i, j] = sym_cxcx_norm[i, j]
#         conn_withHier_norm[i, j] = conn_cxcx_norm[j, i]

plt.figure(figsize=(10,10))
plt.imshow(conn_withHier)
plt.colorbar()
plt.xticks(list(range(n_areas)),area_list,rotation=90,fontname='Georgia',fontsize=7)
plt.yticks(list(range(n_areas)),area_list,fontname='Georgia',fontsize=7)
plt.show()
# conn_6_thres = conn_6 * (conn_6 > threshold_6)
# conn_6_norm_thres = conn_6_norm * (conn_6_norm > threshold_6_norm)


In [ ]:
def getPredHi(areaNo, conn, beta):  # not used anymore
    s1, s2 = 0, 0
    for i in range(n_areas):
        if i in noHierList:
            continue
        s1 += np.exp(beta * conn[areaNo, i]) * hierarchy[i]
        s2 += np.exp(beta * conn[areaNo, i])
    s = s1 / s2
#     h = 0.67 * s + 0.15
    h = 0.73 * s + 0.11
    return h

def getPredHi2(areaNo, conn): 
    s1, s2 = 0, 0
    for i in range(n_areas):
        if i in noHierList:
            continue
        s1 += conn[areaNo, i] * hierarchy[i]
        s2 += conn[areaNo, i]
    s = s1 / s2
    h = 1.33 * s - 0.22
    return h

In [ ]:
import copy
hierarchyEXP = copy.copy(hierarchy)
for i in noHierList:
    hierarchyEXP[i] = getPredHi(i, conn_cxcx, beta=0.7e4)
    hierarchy[i] = getPredHi2(i, conn_cxcx)
    print(hierarchy[i], area_list[i])
hierarchy_df = pd.DataFrame(hierarchy, index=area_list, columns=['hierarchy index'])
# print(hierarchy_df)

In [ ]:
plotHiernotebook(hierarchy_df, 8, r'Hierarchy', (5,3), 12, True, 'loadDataProcessing/hierarchyPlotnotebook.svg')

In [ ]:
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
import brian2
import pickle
import networkx as nx
import copy
import seaborn as sns
from plotFunc import plotPaperTag, getConnHist, getNnodeLoop, plotXYcompr, plotXYcomprCorr, plotXYcomprResidual

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

from adjustText import adjust_text
# import statsmodels.api as sm

In [ ]:
plt.figure(figsize=(5, 3),dpi=300)
plt.scatter(hierarchy, hierarchyEXP)
plt.xlabel('hierarchy(linear)')
plt.ylabel('hierarchy(softmax)')
fontSize = 12
ax = plt.gca()
texts = []
for i in range(len(area_list)):
    acr = area_list[i]
    for div_name, div_color in zip(div_name_list, div_color_list):
        if acr in div[div_name]:
            texts += [ax.text(hierarchy[i], hierarchyEXP[i], acr,
                              color=div_color, fontsize=fontSize*0.4)]

# # use adjust library to adjust the position of annotations.
if True:
    adjust_text(texts, hierarchy, hierarchyEXP,
                ax=ax, precision=0.001,
                arrowprops=dict(arrowstyle='-', color='gray', alpha=.8))

In [ ]:
# from scipy.stats import linregress
# # linregress(a, b)
# i = 8
# print(area_list[i])
# plt.plot(conn_cxcx[i, :]) # input
# plt.figure()
# plt.plot(conn_cxcx[:, i]) # output
# np.exp(1e4*conn_cxcx[i, :]) / sum(np.exp(1e4*conn_cxcx[i, :]))
# np.power(conn_cxcx[i, :], 1/2)

# allhierPred, allhierPred2 = [], []
# for i in range(n_areas):
# #     hPred = sum(np.power(conn_cxcx[i, :], 3) * hierarchy.T.squeeze()) / sum(np.power(conn_cxcx[i, :], 3))
#     hPred = sum(np.exp(1e4 * conn_cxcx[i, :]) * hierarchy.T.squeeze()) / sum(np.exp(1e4 * conn_cxcx[i, :]))
# #     hPred = sum(conn_cxcx[i, :] * hierarchy.T.squeeze()) / sum(conn_cxcx[i, :])
#     hPred2 = sum(conn_cxcx[:, i]**1 * hierarchy.T.squeeze()) / sum(conn_cxcx[:, i]**1)
#     allhierPred.append(hPred)
#     allhierPred2.append(hPred2)
    
# allhierPred = np.array(allhierPred)
# allhierPred2 = np.array(allhierPred2)
# Yhier = hierarchy.T.squeeze()

# plt.figure()
# plt.scatter(allhierPred, Yhier)
# r1, p1 = sp.stats.pearsonr(allhierPred, Yhier)
# lg = linregress(allhierPred, Yhier)
# plt.title(str(r1))
# plt.xlabel('prediction hi')
# plt.ylabel('real hi')
# plt.xlim(0, 1)


# plt.figure()
# plt.scatter(allhierPred2, Yhier)
# r, p = sp.stats.pearsonr(allhierPred2, Yhier)
# plt.title(str(r))
# plt.xlabel('prediction hi')
# plt.ylabel('real hi')
# plt.xlim(0, 1)

In [ ]:
# make plot using densityNorm connectivity. 

In [ ]:
sym_cxcx = 0.5 * (conn_cxcx + conn_cxcx.T)
anti_cxcx = 0.5 * (conn_cxcx - conn_cxcx.T)
sym = (norm(sym_cxcx) - norm(anti_cxcx)) / (norm(sym_cxcx) + norm(anti_cxcx))
print(sym) # if sym ==1 or -1, then conn_cxcx is symmetric. 
px.imshow(sym_cxcx, color_continuous_scale='viridis')
# conn_cxcx_norm
sym_cxcx_norm = 0.5 * (conn_cxcx_norm + conn_cxcx_norm.T)
px.imshow(sym_cxcx_norm, color_continuous_scale='viridis')

In [ ]:
plt.figure()
plt.hist(sym_cxcx[sym_cxcx != 0], bins=np.logspace(np.log10(1e-7),np.log10(1e-2), 50), log=True)
ax = plt.gca()
ax.set_xscale('log')
# plt.figure()
# plt.hist(conn_cxcx[conn_cxcx != 0], bins=np.logspace(np.log10(1e-7),np.log10(1e-2), 50), log=True)
# ax = plt.gca()
# ax.set_xscale('log')
# plt.figure()
# plt.hist(conn_cxcx_norm[conn_cxcx_norm != 0], 
#          bins=np.logspace(np.log10(1e-7),np.log10(1e-2), 50), log=True)
# ax = plt.gca()
# ax.set_xscale('log')
plt.figure()
plt.hist(sym_cxcx_norm[sym_cxcx_norm != 0], 
         bins=np.logspace(np.log10(1e-7),np.log10(1e-2), 50), log=True)
ax = plt.gca()
ax.set_xscale('log')


# use the percentile

print(np.percentile(sym_cxcx, 70))
print(np.percentile(sym_cxcx_norm, 65))

In [ ]:
# generate a graph for the connectivity
# https://github.com/rweng18/midsummer_network
conn_graph = nx.Graph()
for area in area_list_sort:
    conn_graph.add_node(area, size = 2)
conn_graph.nodes()

In [ ]:
# thresholding the network
threshold = 3.21e-4
conn_cxcx_thres = sym_cxcx * (sym_cxcx > threshold)
# add edges to the graph
for area1_index in range(len(area_list_sort)):
    for area2_index in range(len(area_list_sort)):
        if area1_index > area2_index:
            continue
        area1 = area_list_sort[area1_index]
        area2 = area_list_sort[area2_index]
        conn = conn_cxcx_thres[area1_index, area2_index]
        if conn > 0:
            conn_graph.add_edge(
            area1, area2, weight = conn)
  

In [ ]:
# generate position for each node

pos_ = nx.spring_layout(conn_graph)
# 
brainCord = pd.read_csv('brainmapCord.csv')

c1 = np.cos(-0.95)
s1 = np.sin(-0.95)

for area in pos_:
    targetRow = brainCord[brainCord['area'] == area]
# print(targetRow)
    xcord = np.float(targetRow['xcord'])
    ycord = np.float(targetRow['ycord'])
    xcord_ = xcord * c1 - ycord * s1
    ycord_ = xcord * s1 + ycord * c1
    pos_[area] = np.array([xcord_, ycord_])

# pos_['AIp'] = [16, 5]
pos_['MOp'][1] = -5

In [ ]:
def make_edge(x, y, text, width):
    
    '''Creates a scatter trace for the edge between x's and y's with given width

    Parameters
    ----------
    x    : a tuple of the endpoints' x-coordinates in the form, tuple([x0, x1, None])
    
    y    : a tuple of the endpoints' y-coordinates in the form, tuple([y0, y1, None])
    
    width: the width of the line

    Returns
    -------
    An edge trace that goes between x0 and x1 with specified width.
    '''
    return  go.Scatter(x         = x,
                       y         = y,
                       line      = dict(width = width,
                                   color = 'cornflowerblue'),
                       opacity = 0.5,
                       hoverinfo = 'text',
                       text      = ([text]),
                       mode      = 'lines')

In [ ]:
def weight2thick(wgt, threshold):
    return np.log10(wgt) - np.log10(threshold)

In [ ]:
# For each edge, make an edge_trace, append to list

edge_trace = []
for edge in conn_graph.edges():
    wgt = conn_graph.edges()[edge]['weight']
    if wgt > 0:
        area1 = edge[0]
        area2 = edge[1]

        x0, y0 = pos_[area1]
        x1, y1 = pos_[area2]

        text   = area1 + '--' + area2 + ': ' + str(wgt)
        
        trace  = make_edge(
            [x0, x1, None], [y0, y1, None], text,
            weight2thick(wgt, threshold) * 2 + 0.5)
        edge_trace.append(trace)

In [ ]:
 # Make a node trace
node_trace = go.Scatter(x         = [],
                        y         = [],
                        text      = [],
                        textposition = "top center",
                        textfont_size = 5,
                        mode      = 'markers+text',
                        hoverinfo = 'none',
                        marker    = dict(color = [],
                                         size  = [],
                                         line  = None))
# For each node, get the position and size and add to the node_trace
for node in conn_graph.nodes():
    x, y = pos_[node]
    node_trace['x'] += tuple([x])
    node_trace['y'] += tuple([y])
    node_trace['marker']['color'] += tuple(['cornflowerblue'])
    node_trace['marker']['size'] += tuple([5*conn_graph.nodes()[node]['size']])
    node_trace['text'] += tuple(['<b>' + node + '</b>'])

In [ ]:
# get the range for thresholding the network.
a  = sym_cxcx.reshape(-1,1)
plt.hist(a, bins = 100)
print(sum(a > 4.2e-4)/len(a)) 


conn_graph = nx.Graph()
for area in area_list_sort:
    conn_graph.add_node(area, size = 2)
conn_graph.nodes()

# thresholding the network
threshold = 2.9e-4
conn_cxcx_thres = sym_cxcx * (sym_cxcx > threshold)
# add edges to the graph
for area1_index in range(len(area_list_sort)):
    for area2_index in range(area1_index+1, len(area_list_sort)):
        area1 = area_list_sort[area1_index]
        area2 = area_list_sort[area2_index]
        conn = conn_cxcx_thres[area1_index, area2_index]
        if conn > 0:
            conn_graph.add_edge(
            area1, area2, weight = conn)

# For each edge, make an edge_trace, append to list

edge_trace = []
for edge in conn_graph.edges():
    wgt = conn_graph.edges()[edge]['weight']
    if wgt > 0:
        area1 = edge[0]
        area2 = edge[1]

        x0, y0 = pos_[area1]
        x1, y1 = pos_[area2]

        text   = area1 + '--' + area2 + ': ' + str(wgt)
        
        trace  = make_edge(
            [x0, x1, None], [y0, y1, None], text,
            weight2thick(wgt, threshold) * 2 + 0.5)
        edge_trace.append(trace)
        
# Make a node trace
node_trace = go.Scatter(x         = [],
                        y         = [],
                        text      = [],
                        textposition = "top center",
                        textfont_size = 5,
                        mode      = 'markers+text',
                        hoverinfo = 'none',
                        marker    = dict(color = [],
                                         size  = [],
                                         line  = None))
# For each node, get the position and size and add to the node_trace
for node in conn_graph.nodes():
    x, y = pos_[node]
    node_trace['x'] += tuple([x])
    node_trace['y'] += tuple([y])
    node_trace['marker']['color'] += tuple(['cornflowerblue'])
    node_trace['marker']['size'] += tuple([5*conn_graph.nodes()[node]['size']])
    node_trace['text'] += tuple(['<b>' + node + '</b>'])        


layout = go.Layout(
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)'
)

saveFig = True
fileName = 'rawConnectivityMap.svg'

fig = go.Figure(layout = layout)

for trace in edge_trace:
    fig.add_trace(trace)

fig.add_trace(node_trace)

fig.update_layout(showlegend = False)

fig.update_xaxes(showticklabels = False, automargin=True)

fig.update_yaxes(showticklabels = False, scaleanchor='x', scaleratio=1, automargin=True)
fig.update_layout(width = 700, height=580)
fig.show()
if saveFig:
    fig.write_image("figure/loadDataProcessing/"+ fileName, width=700, height=580, scale=1)
# py.plot(fig, filename='conn_graph.html')

In [ ]:
conn_graph = nx.Graph()
for area in area_list_sort:
    conn_graph.add_node(area, size = 2)
conn_graph.nodes()

# thresholding the network
threshold = 4.2e-4 #2.9e-4
conn_cxcx_thres = sym_cxcx * (sym_cxcx > threshold)
# add edges to the graph
for area1_index in range(len(area_list_sort)):
    for area2_index in range(area1_index+1, len(area_list_sort)):
        area1 = area_list_sort[area1_index]
        area2 = area_list_sort[area2_index]
        conn = conn_cxcx_thres[area1_index, area2_index]
        if conn > 0:
            conn_graph.add_edge(
            area1, area2, weight = conn)

# For each edge, make an edge_trace, append to list

edge_trace = []
for edge in conn_graph.edges():
    wgt = conn_graph.edges()[edge]['weight']
    if wgt > 0:
        area1 = edge[0]
        area2 = edge[1]

        x0, y0 = pos_[area1]
        x1, y1 = pos_[area2]

        text   = area1 + '--' + area2 + ': ' + str(wgt)
        
        trace  = make_edge(
            [x0, x1, None], [y0, y1, None], text,
            weight2thick(wgt, threshold) * 2 + 0.5)
        edge_trace.append(trace)
        
# Make a node trace
node_trace = go.Scatter(x         = [],
                        y         = [],
                        text      = [],
                        textposition = "top center",
                        textfont_size = 8,
                        mode      = 'markers+text',
                        hoverinfo = 'none',
                        marker    = dict(color = [],
                                         size  = [],
                                         line  = None))
# For each node, get the position and size and add to the node_trace
for node in conn_graph.nodes():
    x, y = pos_[node]
    node_trace['x'] += tuple([x])
    node_trace['y'] += tuple([y])
    node_trace['marker']['color'] += tuple(['cornflowerblue'])
    node_trace['marker']['size'] += tuple([5*conn_graph.nodes()[node]['size']])
    node_trace['text'] += tuple(['<b>' + node + '</b>'])        


layout = go.Layout(
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)'
)

saveFig = True
fileName = 'rawConnectivityMap.svg'

fig = go.Figure(layout = layout)

for trace in edge_trace:
    fig.add_trace(trace)

fig.add_trace(node_trace)

fig.update_layout(showlegend = False)

fig.update_xaxes(showticklabels = False, automargin=True)

fig.update_yaxes(showticklabels = False, scaleanchor='x', scaleratio=1, automargin=True)
fig.update_layout(width = 700, height=580)
fig.show()
if saveFig:
    fig.write_image("figure/loadDataProcessing/"+ fileName, width=700, height=580, scale=1)
# py.plot(fig, filename='conn_graph.html')

In [ ]:
# get the range for thresholding the network.
a  = sym_cxcx_norm.reshape(-1,1)
plt.hist(a, bins = 100)
print(sum(a > 3.5e-4)/len(a)) 

In [ ]:
# generate a graph for the connectivity
# https://github.com/rweng18/midsummer_network

saveFig = True
fileName = 'connectivityNormalized.svg'

conn_graph = nx.Graph()
for area in area_list_sort:
    conn_graph.add_node(area, size = 2)
conn_graph.nodes()

# thresholding the network
threshold = 3.5e-4  #1.6e-4
conn_cxcx_norm_thres = sym_cxcx_norm * (sym_cxcx_norm > threshold)
# add edges to the graph
for area1_index in range(len(area_list_sort)):
    for area2_index in range(area1_index+1, len(area_list_sort)):

        area1 = area_list_sort[area1_index]
        area2 = area_list_sort[area2_index]
        conn = conn_cxcx_norm_thres[area1_index, area2_index]
        if conn > 0:
            conn_graph.add_edge(
            area1, area2, weight = conn)
            
# For each edge, make an edge_trace, append to list

edge_trace = []
for edge in conn_graph.edges():
    wgt = conn_graph.edges()[edge]['weight']
    if wgt > 0:
        area1 = edge[0]
        area2 = edge[1]

        x0, y0 = pos_[area1]
        x1, y1 = pos_[area2]

        text   = area1 + '--' + area2 + ': ' + str(wgt)
        
        trace  = make_edge(
            [x0, x1, None], [y0, y1, None], text,
            weight2thick(wgt, threshold) * 2 + 0.5)
        edge_trace.append(trace)
        
        
# Make a node trace
node_trace = go.Scatter(x         = [],
                        y         = [],
                        text      = [],
                        textposition = "top center",
                        textfont_size = 8,
                        mode      = 'markers+text',
                        hoverinfo = 'none',
                        marker    = dict(color = [],
                                         size  = [],
                                         line  = None))
# For each node, get the position and size and add to the node_trace
for node in conn_graph.nodes():
    x, y = pos_[node]
    node_trace['x'] += tuple([x])
    node_trace['y'] += tuple([y])
    node_trace['marker']['color'] += tuple(['cornflowerblue'])
    node_trace['marker']['size'] += tuple([5*conn_graph.nodes()[node]['size']])
    node_trace['text'] += tuple(['<b>' + node + '</b>'])   
    
    
    

    layout = go.Layout(
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)'
)



fig = go.Figure(layout = layout)

for trace in edge_trace:
    fig.add_trace(trace)

fig.add_trace(node_trace)

fig.update_layout(showlegend = False)

fig.update_xaxes(showticklabels = False)

fig.update_yaxes(showticklabels = False, scaleanchor='x', scaleratio=1)
fig.update_layout(width = 700, height=580)
fig.show()
if saveFig:
    fig.write_image("figure/loadDataProcessing/"+ fileName, width=700, height=580, scale=1)
# py.plot(fig, filename='conn_graph.html')

In [ ]:
inDegree, outDegree = [], []
thresIn, thresOut = 2.75e-4, 2.75e-4
for i in range(len(area_list)):
    inDegree.append(sum(conn_cxcx_norm[i, :] > thresIn))
    outDegree.append(sum(conn_cxcx_norm[:, i] > thresOut))
df = pd.DataFrame({'area': area_list, 'inDegree': inDegree, 'outDegree': outDegree})
fig = px.line(df, 'area', ['inDegree', 'outDegree'])
fig.update_yaxes(title='degree')
fig.update_layout(legend_title_text='')
fig.show()
fig.write_image("figure/loadDataProcessing/"+ "inOutDegree.svg", width=1000, height=380, scale=1)

In [ ]:
# use 75% to determine threshold.
print(np.percentile(sym_cxcx_norm, 75))
print(np.percentile(sym_cxcx, 75))

In [ ]:
def makeHeatPlotRect(mat, xTicks, yTicks, Xlabel, Ylabel, cbarlabel, colormap, figureSize, fontSize, saveFig, fileName):
    fig = plt.figure(figsize=figureSize, dpi=300, facecolor='w', edgecolor='k')
    plt.rcParams.update({'font.size': fontSize})
#     plt.imshow(mat, colormap, norm=TwoSlopeNorm(0))
    plt.imshow(mat, colormap)
    cbar = plt.colorbar()
    cbar.set_label(cbarlabel)
    plt.xticks(range(len(xTicks)), xTicks, rotation=90, fontsize=fontSize*0.4)
    plt.yticks(range(len(yTicks)), yTicks, fontsize=fontSize*0.4)
    plt.xlabel(Xlabel)
    plt.ylabel(Ylabel)
    
    # save figure
    if saveFig:
        plt.savefig('figure/' + fileName, dpi=300, bbox_inches='tight', transparent=True)
    # plt.savefig('figure/neurondensity_grad.png',dpi=80,bbox_inc

In [ ]:
# rearrange the connnectivity to be hierarhchical. 
sortedAreaList = [area_list[i] for i in np.argsort(list(hierarchy_df['hierarchy index']))]
print(sortedAreaList[:5], len(sortedAreaList))

sortedConnPre = np.array( [conn_cxcx_norm[:,j].tolist() for j in np.argsort(list(hierarchy_df['hierarchy index']))] )
sortedConn = np.array( [sortedConnPre[:,i].tolist() for i in np.argsort(list(hierarchy_df['hierarchy index']))] )

print(sortedConn)

makeHeatPlotRect(sortedConn, sortedAreaList, sortedAreaList,
                'Source', 'Target', 'Connectivity', 'viridis', (5, 3), 12, True, 'loadDataProcessing/cortexConnectivityMatrix.svg')

In [ ]:
sortedconncxth = np.array( [conn_cxth[:,j].tolist() for j in np.argsort(list(hierarchy_df['hierarchy index']))] ).T
# sortedConn = np.array( [sortedConnPre[:,i].tolist() for i in np.argsort(list(hierarchy_df['hierarchy index']))] )
# TODO: check the order of the connectivity
#  unfinished sorted CXTH connections. 
plt.figure(figsize=(10,10))
plt.imshow(conn_cxth)
plt.xticks(range(len(area_list)), area_list, rotation=90)
plt.yticks(range(len(thal_acr)), thal_acr)
plt.xlabel('source')
plt.ylabel('target')
plt.colorbar()

makeHeatPlotRect(sortedconncxth, sortedAreaList, thal_acr,
                'Source', 'Target', 'Connectivity', 'viridis', (5, 3), 12, True, 'loadDataProcessing/cxthConnectivityMatrix.svg')





In [ ]:
sortedconnthcx = np.array( [conn_thcx[i,:].tolist() for i in np.argsort(list(hierarchy_df['hierarchy index']))] )

plt.figure(figsize=(10,10))
plt.imshow(conn_thcx)
plt.yticks(range(len(area_list)), area_list)
plt.xticks(range(len(thal_acr)), thal_acr, rotation=90)
plt.xlabel('source')
plt.ylabel('target')
plt.colorbar()

makeHeatPlotRect(sortedconnthcx, thal_acr, sortedAreaList,
                'Source', 'Target', 'Connectivity', 'viridis', (5, 3), 12, True, 'loadDataProcessing/thcxConnectivityMatrix.svg')


In [ ]:
# save arealist
np.savetxt("results/area_list.csv", sortedAreaList, delimiter=",", fmt = '%s')
# check if loading area list works
arr = np.loadtxt('results/area_list.csv', delimiter=',', dtype='str')
arr[0:5]

In [ ]:
plt.imshow(sortedConn)
plt.colorbar()

In [ ]:
# save connectivity
np.savetxt("results/conn_cxcx.csv", sortedConn, delimiter=",")
np.savetxt("results/conn_cxth.csv", sortedconncxth, delimiter=",")
np.savetxt("results/conn_thcx.csv", sortedconnthcx, delimiter=",")

np.savetxt("results/conn_cxcx_raw.csv", conn_cxcx, delimiter=",")


volume_df.to_csv('results/dfVolume.csv')
density_df.to_csv("results/dfDensity.csv")

# np.savetxt("results/dfDensity.csv", density_df, delimiter=",")

In [ ]:
arr = np.loadtxt('results/conn_cxcx.csv', delimiter=',')
print(arr[0:5, 0:5])
print(arr.shape)

In [ ]:
# load interneuron density 
# and add processing afterwards. 

interneuronfilename = 'mouse_interneuron_density.pkl'
celldensityfilename = 'cell_density_iso.csv'
# get density for related old areas (the area name is based on CCF v2, not latest CCF v3)
old_areas = ['ACAd', 'ACAv', 'AId', 'AIp', 'AIv', 'AUDd', 'AUDp','AUDpo', 'AUDv', 'ECT', 'GU',
'ILA', 'MOp', 'MOs', 'ORBl', 'ORBm', 'ORBvl', 'PERI', 'PL', 'PTLp', 'RSPagl', 'RSPd',
'RSPv', 'SSp-bfd', 'SSp-ll', 'SSp-m', 'SSp-n', 'SSp-tr', 'SSp-ul', 'SSs', 'TEa', 'VISC', 
'VISal', 'VISam', 'VISl', 'VISp', 'VISpl', 'VISpm']

neurons_sorted = load_celldensity(libfilepath, celldensityfilename, old_areas)
 
[PVgrad_df, SSTgrad_df, normPVgrad_df, normSSTgrad_df] = load_interneurondensity(
    libfilepath, interneuronfilename, old_areas)

# modify cell density
# neurons_sorted[37] = neurons_sorted[33] # VISpm seems outlier, replace it with VISam

# load pandas dataframe "results/dfDensity.csv"
density_df = pd.read_csv("results/dfDensity.csv", index_col=0)

In [ ]:
a = np.array(list(PVgrad_df['raw PV density']))
r1 = (a - min(a)) / (max(a) - min(a))
r2 = a / max(a)
plt.scatter(r1, r2)

In [ ]:
len(neurons_sorted)
print(old_areas)

In [ ]:

PVcellNorm = np.array(PVgrad_df)[:, 0] / neurons_sorted
PVcell_df = pd.DataFrame(PVcellNorm, columns=['raw PV density'], index=old_areas)
PVnormcellNorm = np.array(normPVgrad_df)[:, 0] / neurons_sorted 
PVnormcellNorm = PVnormcellNorm / PVnormcellNorm.max()
normPVcell_df = pd.DataFrame(PVnormcellNorm, columns=['norm PV gradient'], index=old_areas)

# SSTcell_df is the SST cell fraction. 
SSTcellNorm = np.array(SSTgrad_df)[:, 0] / neurons_sorted
SSTcell_df = pd.DataFrame(SSTcellNorm, columns=['raw SST density'], index=old_areas)
SSTcell_df

In [ ]:
areasInterest = ['SSp-bfd','VISp', 'MOs', 'PL', 'ILA']
print([PVcell_df['raw PV density'][A] for A in areasInterest])
print([SSTcell_df['raw SST density'][A] for A in areasInterest])

# SSp-bfd VIP: 0.071/2 = 0.0355 total: 0.071+0.056+0.0355 = 0.1625
# VISp VIP: 0.05*0.6 = 0.03 total: 0.051+0.05+0.03 = 0.131
# MOs VIP: 0.056/2 = 0.028   total: 0.056+0.061+0.028 = 0.145
# PL VIP: 0.02*1.3 = 0.026 total: 0.02+0.065+0.026 = 0.111
# ILA VIP: 0.021*0.8 = 0.0168 total: 0.021+0.089+0.0168 = 0.1268


In [ ]:
plt.figure(figsize=(10,8))
plt.plot(neurons_sorted)
plt.xticks(range(len(neurons_sorted)), old_areas, rotation = 90)
# plt.plot(sorted(neurons_sorted))
# sortIndex = np.argsort(neurons_sorted)
# plt.xticks(range(len(neurons_sorted)), [old_areas[i] for i in sortIndex], rotation = 90)
plt.ylabel('neuron density')
plt.ylim((0, 160000))
axes = plt.gca()
axes.spines['top'].set_visible(False)
axes.spines['right'].set_visible(False)
print(sorted(neurons_sorted)[-1] / sorted(neurons_sorted)[0], sorted(neurons_sorted)[-2] / sorted(neurons_sorted)[0])


In [ ]:
# preprocessing the interneuron density
[PVgrad_df, SSTgrad_df, PVcell_df, normPVgrad_df, normSSTgrad_df, normPVcell_df] = interneuronDensityProcessing(
    [PVgrad_df, SSTgrad_df, PVcell_df, normPVgrad_df, normSSTgrad_df, normPVcell_df])

In [ ]:
len(PVcell_df)

In [ ]:
def plotRawDensitynotebook(grad_df, dotSize, yaxis, figureSize, fontSize, saveFig, fileName):
        # seaborn.set()
    dfRaw = grad_df.sort_values(grad_df.columns[0])

    fig = plt.figure(figsize = figureSize, dpi=300, facecolor='w', edgecolor='k')
    plt.rcParams.update({'font.size': fontSize})
    
    xx = np.arange(len(grad_df))
    plt.plot(xx, dfRaw.values, '.-', markersize=dotSize, color='k')
    plt.xticks(xx, dfRaw.index, rotation = 90, fontsize=fontSize*0.6)
    plt.ylabel(yaxis) #r'$e_i/e_1$'
    # regPV = sk.linear_model.LinearRegression()
    # xx2 = xx[np.newaxis].T
    # regPV.fit(xx2, dfRaw.values)
    # print(regPV.coef_, regPV.intercept_)
    # plt.plot(xx, xx * regPV.coef_[0][0] + regPV.intercept_[0], alpha = 0.5)
    ax = plt.gca()
#     [t.set_color('red') for t in ax.xaxis.get_ticklabels()]
    for t in ax.xaxis.get_ticklabels():
        label = t._text
        for i in range(6):
            if label in div[div_name_list[i]]:
                t.set_color(div_color_list[i])
    # plt.grid(alpha = 0.5)

    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    if saveFig:
        plt.savefig('figure/' + fileName, dpi=300, bbox_inches='tight',transparent=True)

In [ ]:
def plotDensityBasedOnRawPVnotebook(grad_df, dotSize, yaxis, figureSize, fontSize, saveFig, fileName):
        # seaborn.set()
    dfSort = grad_df.sort_values(by = 'raw PV density')
    

    fig = plt.figure(figsize = figureSize, dpi=300, facecolor='w', edgecolor='k')
    plt.rcParams.update({'font.size': fontSize})
    
    xx = np.arange(len(grad_df))
    plt.plot(xx, dfSort['neuron density'], '.-', markersize=dotSize, color='k')
    plt.xticks(xx, dfSort['area'], rotation = 90, fontsize=fontSize*0.6)
    plt.ylabel(yaxis) #r'$e_i/e_1$'
    # regPV = sk.linear_model.LinearRegression()
    # xx2 = xx[np.newaxis].T
    # regPV.fit(xx2, dfRaw.values)
    # print(regPV.coef_, regPV.intercept_)
    # plt.plot(xx, xx * regPV.coef_[0][0] + regPV.intercept_[0], alpha = 0.5)
    ax = plt.gca()
#     [t.set_color('red') for t in ax.xaxis.get_ticklabels()]
    for t in ax.xaxis.get_ticklabels():
        label = t._text
        for i in range(6):
            if label in div[div_name_list[i]]:
                t.set_color(div_color_list[i])
    # plt.grid(alpha = 0.5)

    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    if saveFig:
        plt.savefig('figure/' + fileName, dpi=300, bbox_inches='tight',transparent=True)

In [ ]:
# plot the cell density according to the order of raw PV density. 


In [ ]:
plotRawDensitynotebook(PVgrad_df, 8, r'PV cell density (cells/$mm^{3}$)', (5,3), 12, True, 'loadDataProcessing/rawPVnotebook.svg')

In [ ]:
plotRawDensitynotebook(PVcell_df, 8, r'PV cell fraction', (5,3), 12, True, 'loadDataProcessing/ratioPVnotebook.svg')

In [ ]:
# make a dataframe for neurondensity in density_df and raw pv density in PVgrad_df.
density_rawPV_df = pd.DataFrame({'area': list(density_df['area']), 'neuron density': list(density_df['neuron density']), 'raw PV density': list(PVgrad_df['raw PV density'])})
density_rawPV_df.head()

In [ ]:
# sort the dataframe by raw PV density
dfSort = density_rawPV_df.sort_values(by = 'raw PV density')
dfSort  

In [ ]:
plotDensityBasedOnRawPVnotebook(density_rawPV_df, 8, r'Neuron density (cells/$mm^{3}$)', (5,3), 12, True, 'loadDataProcessing/densityBasedOnPVnotebook.svg')

In [ ]:
# plot the neuron density with same order as pv cell density.
def plotRawDensitynotebook(grad_df, dotSize, yaxis, figureSize, fontSize, saveFig, fileName):
    
    dfRaw = grad_df.sort_values(grad_df.columns[0])

    fig = plt.figure(figsize = figureSize, dpi=300, facecolor='w', edgecolor='k')
    plt.rcParams.update({'font.size': fontSize})
    
    xx = np.arange(len(grad_df))
    plt.plot(xx, dfRaw.values, '.-', markersize=dotSize, color='k')
    plt.xticks(xx, dfRaw.index, rotation = 90, fontsize=fontSize*0.6)
    plt.ylabel(yaxis) #r'$e_i/e_1$'
    ax = plt.gca()
    for t in ax.xaxis.get_ticklabels():
        label = t._text
        for i in range(6):
            if label in div[div_name_list[i]]:
                t.set_color(div_color_list[i])
    # plt.grid(alpha = 0.5)

    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    if saveFig:
        plt.savefig('figure/' + fileName, dpi=300, bbox_inches='tight',transparent=True)

In [ ]:
# # plot pv norm by l23 density
# cellnormPVgrad_df = pd.DataFrame(np.array(PVgrad_df['raw PV density']) / np.array(l23density_df['l23 neuron density']), index = area_list, columns = ['cell norm PV density'])
# plotRawDensitynotebook(cellnormPVgrad_df, 8, r'cellnormPV cell density (cells/$mm^{3}$)', (5,3), 12, False, 'loadDataProcessing/cellnormPVnotebook.svg')

In [ ]:
# inhcellnormPVgrad_df = pd.DataFrame(np.array(PVgrad_df['raw PV density']) / np.array(density_df['inhibitory neuron density']), index = area_list, columns = ['Inhibitory cell norm PV density'])
# plotRawDensitynotebook(inhcellnormPVgrad_df, 8, r'inhibitory cell norm PV density (cells/$mm^{3}$)', (5,3), 12, False, 'loadDataProcessing/inhcellnormPVnotebook.svg')

In [ ]:
plotRawDensitynotebook(SSTgrad_df, 8, r'SST cell density (cells/$mm^{3}$)', (5,3), 12, True, 'loadDataProcessing/rawSSTnotebook.svg')

In [ ]:

dfSST = SSTgrad_df.sort_values(SSTgrad_df.columns[0])

plt.figure(figsize = (10, 7))
xx = np.arange(len(dfSST))
plt.plot(xx, dfSST.values)
plt.xticks(xx, dfSST.index, rotation = 90, size = 15)
regSST = sk.linear_model.LinearRegression()
xx2 = xx[np.newaxis].T
regSST.fit(xx2, dfSST.values)
print(regSST.coef_, regSST.intercept_)
plt.plot(xx, xx * regSST.coef_[0][0] + regSST.intercept_[0], alpha = 0.5)

plt.ylabel('SST density (cells/' + r'$mm^{3}$' + ')')
plt.grid(alpha = 0.5)
axes = plt.gca()
axes.spines['top'].set_visible(False)
axes.spines['right'].set_visible(False)

In [ ]:
normPVcell_df

In [ ]:
# generate sorted dataframe of PV density.

sortednormPV_df = pd.DataFrame( [normPVcell_df['norm PV gradient'][i] for i in np.argsort(list(hierarchy_df['hierarchy index'])) ], index=sortedAreaList, columns=['norm PV gradient'])
sortednormPV_df.head()

In [ ]:
sortedrawPV_df = pd.DataFrame( [PVcell_df['raw PV density'][i] for i in np.argsort(list(hierarchy_df['hierarchy index'])) ], index=sortedAreaList, columns=['raw PV relative density'])
sortedrawPV_df.head()

In [ ]:
# save interneuron density
sortednormPV_df.to_csv('results/dfPV.csv')
sortedrawPV_df.to_csv('results/dfPVraw.csv')

# normPVgrad_df.to_csv('results/dfPV.csv')
# normSSTgrad_df.to_csv('results/dfSST.csv')
# PVgrad_df.to_csv('results/dfPVraw.csv')
# SSTgrad_df.to_csv('results/dfSSTraw.csv')

In [ ]:
dfPV = pd.read_csv('results/dfPV.csv', header = 0, index_col=0)
print(len(dfPV))
dfPV.head()



In [ ]:
# generate sorted dataframe of hierarchy.

sortedHierarchy_df = pd.DataFrame( [hierarchy_df['hierarchy index'][i] for i in np.argsort(list(hierarchy_df['hierarchy index'])) ], index=sortedAreaList, columns=['hierarchy index'])
sortedHierarchy_df.head()

In [ ]:
sortedHierarchy_df.to_csv('results/dfHier.csv')

In [ ]:
# load thalamic hierarchy and save dataframe

THhierfilename = 'hier_th_extracted_CC_TC_CT_norm.csv'

with open('lib/' + THhierfilename, 'rb') as f:
    THhier_df = pd.read_csv(f, sep=',', names=['area', 'THhier'], header=0)
THhier_df = THhier_df.drop([34]) # remove RT area  reticular thalamic area  need to change
THhier_df = THhier_df.reset_index(drop=True)
print(np.array(THhier_df['THhier']).shape)
THhier_df = pd.DataFrame(np.array(THhier_df['THhier']), columns=['THhier'], index=np.array(THhier_df['area']))

In [ ]:
ta = np.array(THhier_df).squeeze()
print(ta.shape)


In [ ]:
plt.figure(figsize = (10, 7))
plt.scatter(range(len(thal_acr)), ta[np.argsort(ta)])
plt.xticks(range(len(thal_acr)), np.array(thal_acr)[np.argsort(ta)], rotation = 90, size = 15)
plt.ylabel('Thalamic hierarchy index')

In [ ]:
list(np.array(thal_acr)[np.argsort(ta)]).index(
    'AD')

In [ ]:
THhier_df.to_csv('results/dfTHhier.csv')

In [ ]:
# design using thalamocortical conections to generate a thalamic hierarchy
plt.imshow(sortedconnthcx)
sortedconnthcx.shape

In [ ]:
THhierRawfilename = 'hier_th_extracted_CC_TC_CT_norm.csv'

with open('lib/' + THhierfilename, 'rb') as f:
    THhierRaw_df = pd.read_csv(f, sep=',', names=['area', 'THhier'], header=0)
THhierRaw_df